In [1]:
!pip install --upgrade gdown
!pip install transformers

In [2]:
import gdown

url = 'https://drive.google.com/drive/folders/1qw05BnA1O-XDgJ50OgNGFSlTa9Kls00j?usp=sharing'
gdown.download_folder(url, quiet=True)

['/content/AAPD/aapd_test.tsv',
 '/content/AAPD/aapd_train.tsv',
 '/content/AAPD/aapd_validation.tsv',
 '/content/AAPD/label_test',
 '/content/AAPD/label_train',
 '/content/AAPD/label_val',
 '/content/AAPD/text_test',
 '/content/AAPD/text_train',
 '/content/AAPD/text_val']

In [3]:
!wget https://gist.githubusercontent.com/ArseniyBolotin/7623835da1631b00fb150bcd5b0d909f/raw/wandb_writer.py -O wandb_writer.py

--2022-02-01 10:09:29--  https://gist.githubusercontent.com/ArseniyBolotin/7623835da1631b00fb150bcd5b0d909f/raw/wandb_writer.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2542 (2.5K) [text/plain]
Saving to: ‘wandb_writer.py’

wandb_writer.py     100%[===================>]   2.48K  --.-KB/s    in 0s      

2022-02-01 10:09:29 (39.2 MB/s) - ‘wandb_writer.py’ saved [2542/2542]



In [4]:
# i need this one
def apply_to_dict_values(dict, f):
    for key, value in dict.items():
        dict[key] = f(value)

In [5]:
BERT_TYPE = 'bert-base-uncased'

In [6]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertConfig

class AAPDDataset(Dataset):
    """AAPD dataset."""

    def __init__(self, path):
        self.path = path
        self.data = pd.read_csv(self.path, sep='\t', header=None)
        self.tokenizer = BertTokenizer.from_pretrained(BERT_TYPE)

    def __len__(self):
        return self.data.shape[0]

    @staticmethod
    def target_to_tensor(target):
        return torch.tensor([float(label) for label in target])

    def __getitem__(self, idx):
        data = self.tokenizer(self.data.iloc[idx, 1], return_tensors="pt", max_length=512, padding="max_length", truncation=True)
        apply_to_dict_values(data, lambda x: x.flatten())
        return data, AAPDDataset.target_to_tensor(self.data.iloc[idx, 0])

In [7]:
train_dataset = AAPDDataset('./AAPD/aapd_train.tsv')
val_dataset = AAPDDataset('./AAPD/aapd_validation.tsv')
test_dataset = AAPDDataset('./AAPD/aapd_test.tsv')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
BATCH_SIZE = 64
N_CLASSES = test_dataset[0][1].shape[0]
N_CLASSES

54

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [10]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [11]:
config = BertConfig.from_pretrained(BERT_TYPE)
config.return_dict = True
bert = BertModel.from_pretrained(BERT_TYPE, config=config)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
import torch.nn as nn

class MultiLabelBert(nn.Module):
    def __init__(self, bert_model, n_classes):
        super(MultiLabelBert, self).__init__()
        self.bert_model = bert_model
        for param in self.bert_model.parameters():
            param.requires_grad = False
        self.n_bert_features = bert_model.pooler.dense.out_features
        self.n_classes = n_classes
        self.dense = nn.Linear(self.n_bert_features, self.n_classes)

    def forward(self, inputs):
        bert_output = self.bert_model(**inputs)
        return self.dense(bert_output.pooler_output)


In [13]:
model = MultiLabelBert(bert, N_CLASSES).to(device)

In [14]:
import torch.optim as optim

LEARNING_RATE = 1e-4

In [ ]:
!pip install wandb
from wandb_writer import WandbWriter

In [35]:
a = []
b = []
batch, target = next(iter(train_dataloader))
a.extend(target.to('cpu').tolist())
apply_to_dict_values(batch, lambda x: x.to(device))
b.extend((torch.sigmoid(model(batch)) > 0.5).type(torch.DoubleTensor).to('cpu').tolist())
len(b), len(b[0])

(64, 54)

In [41]:
from sklearn.metrics import hamming_loss

# print( "Classification report: \n", (classification_report(a, b)))
print( "F1 micro averaging:",(hamming_loss(a, b)))


F1 micro averaging: 0.5619212962962963


In [56]:
import tqdm
from sklearn.metrics import f1_score, hamming_loss

def train_model(
        model,
        train_dataloader,
        val_dataloader,
        optimizer,
        criterion,
        n_epochs,
        wandb_writer,
        wandb_iter_start = 0):

    wandb_iter = wandb_iter_start
    for epoch in range(1, n_epochs+1):
        model.train()
        total_loss = 0.
        all_targets = []
        all_preds = []
        for batch, targets in tqdm.tqdm(train_dataloader, f"Train epoch#{epoch}", leave=False):

            apply_to_dict_values(batch, lambda x: x.to(device))
            targets = targets.to(device)
            logits = model(batch)
            all_targets.extend(targets.to('cpu').tolist())
            all_preds.extend((torch.sigmoid(logits) > 0.5).type(torch.DoubleTensor).to('cpu').tolist())
            optimizer.zero_grad()
            loss = criterion(logits, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            wandb_writer.set_step(wandb_iter)
            wandb_writer.add_scalar("Batch train loss", loss.item())
            wandb_iter += 1

        wandb_writer.add_scalar("Train loss", total_loss / len(train_dataloader))
        wandb_writer.add_scalar("Train F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
        wandb_writer.add_scalar("Train Hamming loss",(hamming_loss(all_targets, all_preds)))


        model.eval()

        with torch.no_grad():
            total_loss = 0.
            all_targets = []
            all_preds = []
            for batch, targets in tqdm.tqdm(val_dataloader, f"Val epoch#{epoch}", leave=False):
                apply_to_dict_values(batch, lambda x: x.to(device))
                targets = targets.to(device)
                logits = model(batch)
                all_targets.extend(targets.to('cpu').tolist())
                all_preds.extend((torch.sigmoid(logits) > 0.5).type(torch.DoubleTensor).to('cpu').tolist())
                loss = criterion(logits, targets)
                total_loss += loss.item()
            
            wandb_writer.add_scalar("Validation loss", total_loss / len(val_dataloader))
            wandb_writer.add_scalar("Validation F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
            wandb_writer.add_scalar("Validation Hamming loss",(hamming_loss(all_targets, all_preds)))


In [53]:
model = MultiLabelBert(bert, N_CLASSES).to(device)
optimizer = optim.Adam(params=[p for p in model.parameters() if p.requires_grad], lr=LEARNING_RATE)
train_model(model, train_dataloader, val_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 10, WandbWriter("MultiLabelBert"))

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Batch train loss,▂▄▁▄█▄▁▄▁▂
steps_per_sec,▃▁▅▂▇▁█▄▃
Batch train loss,0.15048
steps_per_sec,0.68209
